### Script a ser utilizado para retirar informação de um ficheiro genbank armazenado na própria máquina ###

In [5]:
import mysql.connector as SQLC

DataBase = SQLC.connect(
   host ="geo.di.uminho.pt",
   user ="bioinformatica",
   password ="20221207",
    database ="grupo_PRJ"
)
Cursor = DataBase.cursor()

In [6]:
#é necessário introduzir o email (Entrez.email = '')

import re 
from Bio import SeqIO
from Bio import Entrez
Entrez.email = '********@alunos.uminho.pt'

In [7]:
#fonte = open("L42022.1.gb",'r')
fonte = open("L42023.1.gb",'r')
locus = fonte.read()
fonte.close()

In [8]:
record = SeqIO.read("sequence.gb", 'genbank')

### Tabela Locus ###

In [9]:
accession=''
accession_regex = re.search(r'ACCESSION\s+(\w+)', locus)
if accession_regex:
    accession = accession_regex.group(1)

locus_name = ""
locus_name_regex = re.match(r'LOCUS\s+(\w+)', locus)
if locus_name_regex:
    locus_name = locus_name_regex.group(1)

molecule = ''
molecule_regex = re.search(r'LOCUS[\s\w]+\d+\s[bp]{2}\s+(\w+)', locus)
if molecule_regex:
    molecule = molecule_regex.group(1)

segment_type = ''
segment_type_regex = re.search(r'LOCUS[\s\w]+\d+\s[bp]{2}\s+\w+\s+(\w+)', locus)
if segment_type_regex:
    segment_type = segment_type_regex.group(1)

genbank_date = ''
genbank_date_regex = re.search(r'LOCUS[\s\w]+\d+\s[bp]{2}[\s\w]+\w{3}\s+(\d+-[A-Z]+-\d+)', locus)
if genbank_date_regex:
    genbank_date = genbank_date_regex.group(1)

genbank_division = ''
genbank_division_regex = re.search(r'LOCUS[\s\w]+\d+\s[bp]{2}[\s\w]+(\w{3})', locus)
if genbank_division_regex:
    genbank_division = genbank_division_regex.group(1)


sql_locus = f"INSERT INTO locus (accession, locus_name, molecule, segment_type, genbank_date, genbank_division) VALUES ('{accession}', '{locus_name}', '{molecule}', '{segment_type}', '{genbank_date}', '{genbank_division}')"

print(sql_locus)

Cursor.execute(sql_locus)
DataBase.commit()



INSERT INTO locus (accession, locus_name, molecule, segment_type, genbank_date, genbank_division) VALUES ('NM_001198530', 'NM_001198530', 'mRNA', 'linear', '30-DEC-2022', 'PRI')


### Tabela sequences ###

In [10]:
seq = ""
seq_regex = re.findall(r'^\s+\d+ [actg ]+', locus, re.MULTILINE )
if seq_regex:
    for linha in seq_regex:
        m = re.match( r'\s+\d+ (.+)', linha, re.DOTALL )
        seq = seq + re.sub(r'\s+', '', m.group(1) )

seq_length = len(seq)

locus_accession = accession

sql_sequences = f"INSERT INTO sequences (seq, seq_length, locus_accession) VALUES ('{seq.upper()}', {seq_length}, '{locus_accession}')"

print(sql_sequences)

Cursor.execute(sql_sequences)

DataBase.commit()


INSERT INTO sequences (seq, seq_length, locus_accession) VALUES ('GTCAATAATCTCCGCTCCCAGACTACTCCGTTCCTCCGGATTTCGATCCCCCTTTTTCTATCTGTCAATCAGCGCCGCCTTTGAACTGAAAAGCTCTCAGTCTAACTTCAACTCACTCAAATCCGAGCGGCACGAGCACCTCCTGTATCTTCGGCTTCCCCCCCCCTTTGCTCTTTATATCTGACTTCTTGTTGTTGTTGGTGTTTTTTTTTTTTTTACCCCCCTTTTTTATTTATTATTTTTTTGCACATTGATCGGATCCTTGGGAACGAGAGAAAAAAGAAACCCAAACTCACGCGTGCAGAAGATCTCCCCCCCCTTCCCCTCCCCTCCTCCCTCTTTTCCCCTCCCCAGGAGAAAAAGACCCCCAAGCAGAAAAAAGTTCACCTTGGACTCGTCTTTTTCTTGCAATATTTTTTGGGGGGGCAAAACTTTTTGGGGGTGATTTTTTTTGGCTTTTCTTCCTCCTTCATTTTTCTTCCAAAATTGCTGCTGGTGGGTGAAAAAAAAATGCCGCAGCTGAACGGCGGTGGAGGGGATGACCTAGGCGCCAACGACGAACTGATTTCCTTCAAAGACGAGGGCGAACAGGAGGAGAAGAGCTCCGAAAACTCCTCGGCAGAGAGGGATTTAGCTGATGTCAAATCGTCTCTAGTCAATGAATCAGAAACGAATCAAAACAGCTCCTCCGATTCCGAGGCGGAAAGACGGCCTCCGCCTCGCTCCGAAAGTTTCCGAGACAAATCCCGGGAAAGTTTGGAAGAAGCGGCCAAGAGGCAAGATGGAGGGCTCTTTAAGGGGCCACCGTATCCCGGCTACCCCTTCATCATGATCCCCGACCTGACGAGCCCCTACCTCCCCAACGGATCGCTCTCGCCCACCGCCCGAACCTCTAACAAAGTGCCAGTGGTGCAGCACCCTCACCATGTCCACC

### Tabela definition ###


In [11]:
definition = record.description

locus_accession = accession

sql_definition = f"INSERT INTO definition (definition, locus_accession) VALUES ('{definition}', '{locus_accession}')"

print(sql_definition)

Cursor.execute(sql_definition)

DataBase.commit()

INSERT INTO definition (definition, locus_accession) VALUES ('Homo sapiens transcription factor 7 like 2 (TCF7L2), transcript variant 12, mRNA', 'NM_001198530')


### Tabela locus_keywords ###

In [12]:
keyword_dict = {}

keywords_name = record.annotations['keywords']

for num, keyword in enumerate(keywords_name):
    if keywords_name == ['']:
        print('Este ficheiro não tem keywords associadas')
    else:
        sql_locus_keywords = f"INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('{locus_accession}', {num+1}, '{keyword}')"
        Cursor.execute(sql_locus_keywords)
        DataBase.commit()
        print(sql_locus_keywords)

INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('NM_001198530', 1, 'RefSeq')


### Tabela source ###

In [13]:
scientific_name = record.annotations['source']

sql_source = f"INSERT INTO source (scientific_name, locus_accession) VALUES ('{scientific_name}', '{locus_accession}')"

print(sql_source)

Cursor.execute(sql_source)

DataBase.commit()

INSERT INTO source (scientific_name, locus_accession) VALUES ('Homo sapiens (human)', 'NM_001198530')


### Tabela taxonomy ###


In [14]:
taxonomy = record.annotations['taxonomy']

sql_taxonomy = f"INSERT INTO taxonomy (taxonomy, locus_accession) VALUES ('{'; '.join(taxonomy)}', '{locus_accession}')"

print(sql_taxonomy)

Cursor.execute(sql_taxonomy)

DataBase.commit()

INSERT INTO taxonomy (taxonomy, locus_accession) VALUES ('Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi; Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini; Catarrhini; Hominidae; Homo', 'NM_001198530')


### Tabela reference ###

In [15]:
title = list()
journal = list()
pubmed_id = list()
ref_location = list()

try:
    for reference in record.annotations['references']:
        title.append(reference.title)
        journal.append(reference.journal)
        pubmed_id.append(reference.pubmed_id)
        for loc in reference.location:
            ref_location.append(loc)
except KeyError:  # caso não existam referências nos ficheiros --> pass
    pass
    
for i in zip(title, journal, pubmed_id, ref_location):
    if i[2] == '' and i[3] == '':
        sql_references = f'INSERT INTO reference (title, journal, locus_accession) VALUES ("{i[0]}", "{i[1]}", "{locus_accession}")'
        print(sql_references)
        print('Esta referência não possui ID do pubmed, nem descreve a sua localização')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()

    elif i[2] == '' and i[3] != '':
        sql_references = f'INSERT INTO reference (title, journal, locus_accession, ref_location) VALUES ("{i[0]}", "{i[1]}", "{locus_accession}", "{i[3]}")'
        print(sql_references)
        print('Esta referência não possui ID do pubmed')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()

    elif i[2] != '' and i[3] == '':
        sql_references = f'INSERT INTO reference (title, journal, pubmed_id, locus_accession) VALUES ("{i[0]}", "{i[1]}", "{i[2]}", "{locus_accession}")'
        print(sql_references)
        print('Esta referência não descreve a sua localização')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()
        
    else:
        sql_references = f'INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ("{i[0]}", "{i[1]}", "{i[2]}", "{locus_accession}", "{i[3]}")'
        print(sql_references)
        print()
        Cursor.execute(sql_references)
        DataBase.commit()
        

INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ('Milder loss of insulin-containing islets in individuals with type 1 diabetes and type 2 diabetes-associated TCF7L2 genetic variants', 'Diabetologia 66 (1), 127-131 (2023)', '36282337', 'NM_001198530', '[0:3781]')

INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ('TCF7L2 gene associated postprandial triglyceride dysmetabolism- a novel mechanism for diabetes risk among Asian Indians', 'Front Endocrinol (Lausanne) 13, 973718 (2022)', '36263318', 'NM_001198530', '[0:3781]')

INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ('The positive regulatory loop of TCF4N/p65 promotes glioblastoma tumourigenesis and chemosensitivity', 'Clin Transl Med 12 (9), e1042 (2022)', '36116131', 'NM_001198530', '[0:3781]')

INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ('Genotypes and Haplotypes in the

### Tabela pubmed ###

In [16]:
for id in pubmed_id:
    if id != '':
        pubmed_handle = Entrez.esummary(db="pubmed", id=id, retmode="xml")
        pubmed_record = Entrez.read(pubmed_handle)
        pubmed_handle.close()
        for info in pubmed_record:
            title_pubmed = list()
            pub_date = list()
            pub_source = list()
            pub_issue = list()
            pub_pages = list()
            

            title_pubmed.append(info['Title'])
            pub_date.append(info['PubDate'])
            pub_source.append(info['Source'])
            pub_issue.append(info['Issue'])
            pub_pages.append(info['Pages'])
            
        for i in zip(title_pubmed, pub_date, pub_source, pub_issue, pub_pages):
            sql_pubmed = f'INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages) VALUES ({id}, "{i[0]}", "{i[1]}", "{i[2]}", "{i[3]}", "{i[4]}")'
            print(sql_pubmed)    
            Cursor.execute(sql_pubmed)
            DataBase.commit()
            
    else:
        print('Sem resultados')

INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages, doi) VALUES (36282337, 'Milder loss of insulin-containing islets in individuals with type 1 diabetes and type 2 diabetes-associated TCF7L2 genetic variants.', '2023 Jan', 'Diabetologia', '1', '127-131', '10.1007/s00125-022-05818-y')
INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages, doi) VALUES (36263318, 'TCF7L2 gene associated postprandial triglyceride dysmetabolism- a novel mechanism for diabetes risk among Asian Indians.', '2022', 'Front Endocrinol (Lausanne)', '', '973718', '10.3389/fendo.2022.973718')
INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages, doi) VALUES (36116131, 'The positive regulatory loop of TCF4N/p65 promotes glioblastoma tumourigenesis and chemosensitivity.', '2022 Sep', 'Clin Transl Med', '9', 'e1042', '10.1002/ctm2.1042')
INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages, doi) VALUES (35996498, 'Genotypes and Haplotypes in the AXIN2 and TCF7L2 Genes ar

### Tabela authors ###

In [17]:
author_names = dict()

for id in pubmed_id:
    if id != '':
        pubmed_handle = Entrez.esummary(db="pubmed", id=id, retmode="xml")
        pubmed_record = Entrez.read(pubmed_handle)
        pubmed_handle.close()

        for info in pubmed_record:
            author_names[id] = info['AuthorList']
            
        for id, authors in author_names.items():
            for name in authors:
                sql_authors = f"INSERT INTO authors (author_name, pubmed_id) VALUES ('{name}', {id})"
                print(sql_authors)
                Cursor.execute(sql_authors)
                DataBase.commit()
    else:
        print('Sem resultados')


INSERT INTO authors (author_name, pubmed_id) VALUES ('Redondo MJ', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Richardson SJ', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Perry D', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Minard CG', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Carr ALJ', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Brusko T', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Kusmartseva I', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Pugliese A', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Atkinson MA', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Redondo MJ', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Richardson SJ', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Perry D', 36282337)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Minard CG', 36282337)
I

### Tabela feat_source ###

In [18]:
features = record.features

for i in features:
    if i.type == 'source':
        location = set(i for i in i.location)
        organism = set(i for i in i.qualifiers['organism'])
        db_xref = set(i.qualifiers['db_xref'])

        sql_feat_source = f'INSERT INTO feat_source (location, organism, db_xref, locus_accession) VALUES ("{i.location}", "{organism}", "{db_xref}", "{locus_accession}")'
        print(sql_feat_source)
        Cursor.execute(sql_feat_source)
        DataBase.commit()
        


INSERT INTO feat_source (location, organism, db_xref, locus_accession) VALUES ("[0:3781](+)", "{'Homo sapiens'}", "{'taxon:9606'}", "NM_001198530")


### Tabela feat_cds ###

In [19]:
location_list = list()
codon_start_list = list()
product_list = list()
protein_id_list = list()
translation_list = list()

for i in features:
    if i.type == 'CDS':
            
        location_list.append(i.location)
        codon_start_list.append(i.qualifiers['codon_start'])
        protein_id_list.append(i.qualifiers['protein_id'])
        translation_list.append(i.qualifiers['translation'])
        if 'product' in i.qualifiers.keys():
            product_list.append(i.qualifiers['product'])

if 'product' in i.qualifiers.keys():
   for i in zip(location_list, codon_start_list, product_list, protein_id_list, translation_list):
        sql_feat_cds = f'INSERT INTO feat_cds (location, codon_start, product, protein_id, translation, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{i[2][0]}", "{i[3][0]}", "{i[4][0]}", "{locus_accession}")'
        print(sql_feat_cds) 
        Cursor.execute(sql_feat_cds)
        DataBase.commit()
        
else:
    for i in zip(location_list, codon_start_list, protein_id_list, translation_list):
        sql_feat_cds = f'INSERT INTO feat_cds (location, codon_start, protein_id, translation, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{i[2][0]}", "{i[3][0]}", "{locus_accession}")'
        print(sql_feat_cds)
        Cursor.execute(sql_feat_cds)
        DataBase.commit()
        

INSERT INTO feat_cds (location, codon_start, protein_id, translation, locus_accession) VALUES ("[510:1737](+)", "1", "NP_001185459.1", "MPQLNGGGGDDLGANDELISFKDEGEQEEKSSENSSAERDLADVKSSLVNESETNQNSSSDSEAERRPPPRSESFRDKSRESLEEAAKRQDGGLFKGPPYPGYPFIMIPDLTSPYLPNGSLSPTARTSNKVPVVQHPHHVHPLTPLITYSNEHFTPGNPPPHLPADVDPKTGIPRPPHPPDISPYYPLSPGTVGQIPHPLGWLVPQQGQPVYPITTGGFRHPYPTALTVNASMSRFPPHMVPPHHTLHTTGIPHPAIVTPTVKQESSQSDVGSLHSSKHQDSKKEEEKKKPHIKKPLNAFMLYMKEMRAKVVAECTLKESAAINQILGRRWHALSREEQAKYYELARKERQLHMQLYPGWSARDNYGKKKKRKRDKQPGETNGEKKSAFATYKVKAAASAHPLQMEAY", "NM_001198530")


### Tabela feat_gene ###

In [20]:
for i in features:
    if i.type == 'gene':
        
        gene_location_list = list()
        gene_name_list = list()

        gene_location_list.append(i.location)
        
        if 'locus_tag' in i.qualifiers.keys():
            gene_name_list.append(i.qualifiers['locus_tag'])
        elif 'gene' in i.qualifiers.keys():
            gene_name_list.append(i.qualifiers['gene'])

        if 'locus_tag' or 'gene' in i.qualifiers.keys():
            for i in zip(gene_location_list, gene_name_list):
                sql_feat_gene = f'INSERT INTO feat_gene (location, name, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{locus_accession}")'
                print(sql_feat_gene)    
                Cursor.execute(sql_feat_gene)
                DataBase.commit()
                
        else:
            for i in gene_location_list:
                sql_feat_gene = f'INSERT INTO feat_gene (location, locus_accession) VALUES ("{i}", "{locus_accession}")'
                print(sql_feat_gene)    
                Cursor.execute(sql_feat_gene)
                DataBase.commit()
                

INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[0:3781](+)", "TCF7L2", "NM_001198530")
